# Henter type-fane
Tips: https://www.geeksforgeeks.org/how-to-write-pandas-dataframes-to-multiple-excel-sheets/

In [1]:
import pandas as pd
import sqlite3
# Connect to the SQLite database
conn = sqlite3.connect("C:/Projects/nin3-kode-api/NiN3.WebApi/nin3kodeapi.db")
TypefaneQ = """WITH eco AS (
        SELECT Ordinal, Verdi, Beskrivelse 
        FROM Enumoppslag 
        WHERE Enumtype = 'EcosysnivaaEnum'
    )
    SELECT Langkode, Kode, type.Navn, eco.Verdi AS EcosystnivaaEnum
    FROM type
    LEFT JOIN eco ON type.Ecosystnivaa = eco.Ordinal"""  # Write your SQLite query
df_type = pd.read_sql_query(TypefaneQ, conn)  # Execute the query and store the result in a DataFrame
#Ecosystnivå
EcoQ = "Select Verdi, Beskrivelse from Enumoppslag where Enumtype = 'EcosysnivaaEnum'"
df_eco = pd.read_sql_query(EcoQ, conn)
conn.close()

In [3]:
## opprett excel
with pd.ExcelWriter("nin3_0.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    df_type.to_excel(writer, sheet_name="Type", index=False)
    df_eco.to_excel(writer, sheet_name="Ecosystnivå", index=False)
    #data_frame3.to_excel(writer, sheet_name="Baked Items", index=False)
    

In [4]:
from datetime import datetime
import sqlite3
from datetime import datetime

current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
print(current_time)


# Connect to the SQLite database
conn = sqlite3.connect("C:/Projects/nin3-kode-api/NiN3.WebApi/nin3kodeapi.db")

# Get the current timestamp
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

# Insert the values into the db_info table
insert_query = "INSERT INTO db_info (Tittel, Verdi) VALUES (?, ?)"
conn.execute(insert_query, ('Opprettet', current_time))

# Commit the changes and close the connection
conn.commit()
conn.close()
# Connect to the SQLite database
conn = sqlite3.connect("your_database.db")

# Get the current timestamp
current_time = conn.execute("SELECT ").fetchone()[0]
strftime('%Y%m%d_%H%M%S', 'now', 'localtime')
# Print the current time
print(current_time)

# Close the connection
conn.close()


20231129_091209


In [2]:
import os
current_directory = os.getcwd()
print(current_directory)


AttributeError: module 'os' has no attribute 'cwd'